# Tutorial

https://www.youtube.com/watch?v=_C8kWso4ne4

In [1]:
import pyspark
import pandas as pd
import os
import shutil
ROOT = os.getcwd()

In [2]:
imdb1000 = os.path.join(ROOT, 'datasets', 'imdb1000.csv') 
df = pd.read_csv(imdb1000)
print(df.shape)
df.head()

(1000, 6)


,id,title,genres,averageRating,numVotes,releaseYear
0,tt0111161,The Shawshank Redemption,"[""Drama""]",9.3,2951083,1994
1,tt0068646,The Godfather,"[""Crime"", ""Drama""]",9.2,2057179,1972
2,tt0252487,The Chaos Class,"[""Comedy""]",9.2,43570,1975
3,tt0259534,Ramayana: The Legend of Prince Rama,"[""Action"", ""Adventure"", ""Animation""]",9.2,15407,1993
4,tt16747572,The Silence of Swastika,"[""Documentary"", ""History""]",9.2,10567,2021


In [3]:
# Starting spark session 
from pyspark.sql import SparkSession

In [4]:
#
# SparkSession , introduced in spark 2.0
# combines functions: SparkContext, SQLContext, HiveContext, StreamingContext

spark = SparkSession.builder.appName('practice_app').getOrCreate()
spark

24/10/16 14:43:32 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 192.168.1.112 instead (on interface wlo1)
24/10/16 14:43:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 14:43:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Define a schema

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType

schema = StructType([
    StructField(name="id", dataType=StringType(), nullable=True),
    StructField(name="title", dataType=StringType(), nullable=True),
    StructField(name="genres", dataType=ArrayType(elementType=StringType()), nullable=True),
    StructField(name="averageRating", dataType=DoubleType(), nullable=True),
    StructField(name="numVotes", dataType=IntegerType(), nullable=True),
    StructField(name="releaseYear", dataType=IntegerType(), nullable=True),
])


In [6]:
#Read the csv
#escape to tell pyspark that "[""Action"", ""Adventure"", ""Animation""]" is in 1 column
df_spark = spark.read.csv(imdb1000, header = True, inferSchema=True, escape='"')

#specify your own schema
#There's an error with this line
# df_spark = spark.read.csv(imdb1000, header = True, schema=schema, escape='"')


df_spark.show()

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt007156

In [7]:
print(type(df_spark))
df_spark.head(5)

<class 'pyspark.sql.dataframe.DataFrame'>


[Row(id='tt0111161', title='The Shawshank Redemption', genres='["Drama"]', averageRating=9.3, numVotes=2951083, releaseYear=1994),
 Row(id='tt0068646', title='The Godfather', genres='["Crime", "Drama"]', averageRating=9.2, numVotes=2057179, releaseYear=1972),
 Row(id='tt0252487', title='The Chaos Class', genres='["Comedy"]', averageRating=9.2, numVotes=43570, releaseYear=1975),
 Row(id='tt0259534', title='Ramayana: The Legend of Prince Rama', genres='["Action", "Adventure", "Animation"]', averageRating=9.2, numVotes=15407, releaseYear=1993),
 Row(id='tt16747572', title='The Silence of Swastika', genres='["Documentary", "History"]', averageRating=9.2, numVotes=10567, releaseYear=2021)]

In [8]:
df_spark.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)
 |-- releaseYear: integer (nullable = true)



# Indexing

In [9]:
col = df_spark.select('title')
print('.select() returns DataFrame type =' , type(col))
print('Column type ', df_spark['title'])
print("NOTE: Column doesn't have show()")
col.show()

.select() returns DataFrame type = <class 'pyspark.sql.dataframe.DataFrame'>
Column type  Column<'title'>
NOTE: Column doesn't have show()
+--------------------+
|               title|
+--------------------+
|The Shawshank Red...|
|       The Godfather|
|     The Chaos Class|
|Ramayana: The Leg...|
|The Silence of Sw...|
|     The Dark Knight|
|The Lord of the R...|
|    Schindler's List|
|The Godfather Par...|
|        12 Angry Men|
|               Kaiva|
|        Pulp Fiction|
|The Lord of the R...|
|         Mirror Game|
|         Tosun Pasha|
|The Chaos Class F...|
|O.J.: Made in Ame...|
|         The Message|
|           Inception|
|          Fight Club|
+--------------------+
only showing top 20 rows



In [10]:
df_spark.describe().show()

24/10/16 14:43:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+------------------+--------------------+-------------------+-----------------+------------------+
|summary|       id|             title|              genres|      averageRating|         numVotes|       releaseYear|
+-------+---------+------------------+--------------------+-------------------+-----------------+------------------+
|  count|     1000|              1000|                1000|               1000|             1000|              1000|
|   mean|     NULL|1343.6666666666667|                NULL|  8.136899999999924|       276016.383|          1992.287|
| stddev|     NULL|1081.6904979398373|                NULL|0.25383621259155775|427301.1837930772|25.646761732963196|
|    min|tt0010323|             #Home|["Action", "Adven...|                7.8|            10122|              1920|
|    max|tt9900782|             Zübük|         ["Western"]|                9.3|          2951083|              2024|
+-------+---------+------------------+--------------------+-----

# Add column

In [11]:
df_spark = df_spark.withColumn('newCol', df_spark['releaseYear'] - 1000)
df_spark.show()

+----------+--------------------+--------------------+-------------+--------+-----------+------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|newCol|
+----------+--------------------+--------------------+-------------+--------+-----------+------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|   994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|   972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|   975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|   993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|  1021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|  1008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|  1003|
| tt0108052|    Schindler's Li

# Drop col

In [12]:
df_spark = df_spark.drop('newCol')
df_spark.show()

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt007156

# Rename col

In [13]:
df_spark = df_spark.withColumnRenamed('releaseYear', 'year')
df_spark.show()

+----------+--------------------+--------------------+-------------+--------+----+
|        id|               title|              genres|averageRating|numVotes|year|
+----------+--------------------+--------------------+-------------+--------+----+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|1993|
| tt0071562|The Godfather Par...|  ["Crime", "Drama"]|          9.0| 1390249|1974|
| tt

In [14]:
df_spark = df_spark.withColumnRenamed('year', 'releaseYear')
df_spark.show()

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt007156

# Handle missing values

- drop cols
- drop rows
- many parameters
- handle missing values by mean, median, mode

In [15]:
#drop missing values

print('original rows # = ', df_spark.count())
test = df_spark.na.drop()
#another way
# test = df_spark.dropna()
#drop Args:
# - axis: 0 defualt
# - how: 'any', 'all'
# - thresh: int, requires # of NAs to drop
# - inplace
# - subset: list of columns to consider, if not set, consider all columns

print('rows # after dropping NAs' ,test.count())

original rows # =  1000
rows # after dropping NAs 1000


# Fill missing values

In [16]:
df_spark.fillna('Value To Fill', subset=['title', 'genres'])

DataFrame[id: string, title: string, genres: string, averageRating: double, numVotes: int, releaseYear: int]

# Filtering

In [17]:
filtered = df_spark.filter("averageRating >= 9.0")
filtered.show()
print(type(filtered))
print('filtered count = ', filtered.count())

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt007156

In [18]:
# same thing
filtered = df_spark.filter(
    (df_spark['averageRating'] >= 9.0)
      & (df_spark['averageRating'] <= 9.2))
filtered.show()
print(filtered.count())

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt0071562|The Godfather Par...|  ["Crime", "Drama"]|          9.0| 1390249|       1974|
| tt005008

# Apply aggregate and groupby

In [19]:
avg_rating_by_year = df_spark.groupBy('releaseYear').mean()

print('avg_rating_by_year = ', avg_rating_by_year.count())
avg_rating_by_year.show()


avg_rating_by_year =  103
+-----------+------------------+------------------+----------------+
|releaseYear|avg(averageRating)|     avg(numVotes)|avg(releaseYear)|
+-----------+------------------+------------------+----------------+
|       1959| 8.162500000000001|         149644.75|          1959.0|
|       1990|               8.2|          431445.8|          1990.0|
|       1975| 8.315384615384614|235622.76923076922|          1975.0|
|       1977|              8.33|          188227.6|          1977.0|
|       1924| 8.066666666666666|28420.666666666668|          1924.0|
|       2003|  8.12857142857143|          406767.0|          2003.0|
|       2007| 8.000000000000004|         292007.65|          2007.0|
|       2018|             8.224|         163624.52|          2018.0|
|       1974| 8.185714285714285|285517.28571428574|          1974.0|
|       2015| 8.066666666666668|        315840.375|          2015.0|
|       2023|           8.24375|       139384.6875|          2023.0|
|       

In [23]:
#agg by different columns
group_agg = df_spark.groupBy('releaseYear').agg({'averageRating': 'avg', 'numVotes': 'sum'})
group_agg.show()

+-----------+------------------+-------------+
|releaseYear|avg(averageRating)|sum(numVotes)|
+-----------+------------------+-------------+
|       1959| 8.162500000000001|      1197158|
|       1990|               8.2|      2157229|
|       1975| 8.315384615384614|      3063096|
|       1977|              8.33|      1882276|
|       1924| 8.066666666666666|        85262|
|       2003|  8.12857142857143|      8542107|
|       2007| 8.000000000000004|      5840153|
|       2018|             8.224|      4090613|
|       1974| 8.185714285714285|      1998621|
|       2015| 8.066666666666668|      7580169|
|       2023|           8.24375|      2230155|
|       1927|               8.2|       243264|
|       1955| 8.120000000000001|       264981|
|       2006| 8.122222222222224|      7597569|
|       2022| 8.230434782608697|      3518907|
|       1978|              8.34|       456910|
|       1925|               8.0|       182746|
|       1961|            8.0125|       425745|
|       2013|

# Joining 

df1.join(df2, 'col_to_match', 'inner') # inner, outer, full, ...

In [ ]:
# df1.join(df2, 'col_to_match', 'inner') # inner, outer, full, ...

# Sort

.orderBy('averageRating')

pyspark.sql.functions.col
pyspark.sql.functions.desc


# order by priorities, if price is equal, check id

.orderBy( 
    col('price').desc(), 
    col('id').desc() 
)



# Register DF as Table and use SQL 

In [25]:
df_spark.createOrReplaceTempView('imdb') #register df as table

In [26]:
df_spark.show()

+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993|
| tt007156

In [29]:
result = spark.sql('SELECT * FROM imdb WHERE averageRating >= 9.0')
print('count = ', result.count())
result.show()

count =  11
+----------+--------------------+--------------------+-------------+--------+-----------+
|        id|               title|              genres|averageRating|numVotes|releaseYear|
+----------+--------------------+--------------------+-------------+--------+-----------+
| tt0111161|The Shawshank Red...|           ["Drama"]|          9.3| 2951083|       1994|
| tt0068646|       The Godfather|  ["Crime", "Drama"]|          9.2| 2057179|       1972|
| tt0252487|     The Chaos Class|          ["Comedy"]|          9.2|   43570|       1975|
| tt0259534|Ramayana: The Leg...|["Action", "Adven...|          9.2|   15407|       1993|
|tt16747572|The Silence of Sw...|["Documentary", "...|          9.2|   10567|       2021|
| tt0468569|     The Dark Knight|["Action", "Crime...|          9.0| 2932304|       2008|
| tt0167260|The Lord of the R...|["Action", "Adven...|          9.0| 2020203|       2003|
| tt0108052|    Schindler's List|["Biography", "Dr...|          9.0| 1480407|       1993

# Remove temporary view from df

In [30]:
spark.catalog.dropTempView('imdb')

True

# Subqueries

In [31]:
spark.stop()